# Cleaning Data 

## Overview
This notebook cleans the raw dataset (`partizan_2022_raw.csv`) from the 2022-2023 EuroLeague season for the Partizan basketball team. The cleaning process involves loading the data, exploring its structure, handling missing values, standardizing formats, checking for duplicates and inconsistencies, and saving the cleaned data. This ensures the dataset is ready for accurate analysis in subsequent steps.

In [13]:
import pandas as pd

file_path = "../data/partizan_2022_raw.csv"

df = pd.read_csv(file_path)

Understand the dataset’s structure, including the number of entries, column names, data types, and non-null counts, to identify areas needing cleaning.

In [14]:
# exploring the data 
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_player_id          500 non-null    object 
 1   game_id                 500 non-null    object 
 2   game                    500 non-null    object 
 3   round                   500 non-null    int64  
 4   phase                   500 non-null    object 
 5   season_code             500 non-null    object 
 6   player_id               500 non-null    object 
 7   is_starter              500 non-null    float64
 8   is_playing              500 non-null    float64
 9   team_id                 500 non-null    object 
 10  dorsal                  500 non-null    object 
 11  player                  500 non-null    object 
 12  minutes                 498 non-null    object 
 13  points                  500 non-null    int64  
 14  two_points_made         500 non-null    in

Display all column names to confirm the dataset’s contents and identify columns for cleaning or removal.

In [15]:
# exploring columns
print(df.columns)

Index(['game_player_id', 'game_id', 'game', 'round', 'phase', 'season_code',
       'player_id', 'is_starter', 'is_playing', 'team_id', 'dorsal', 'player',
       'minutes', 'points', 'two_points_made', 'two_points_attempted',
       'three_points_made', 'three_points_attempted', 'free_throws_made',
       'free_throws_attempted', 'offensive_rebounds', 'defensive_rebounds',
       'total_rebounds', 'assists', 'steals', 'turnovers', 'blocks_favour',
       'blocks_against', 'fouls_committed', 'fouls_received', 'valuation',
       'plus_minus'],
      dtype='object')


Identify missing values in each column to determine how to handle them for a complete dataset.

In [16]:
# check for missing values 
missing_values_all = df.isnull().sum()
print("Missing values in all columns:")
print(missing_values_all)

Missing values in all columns:
game_player_id            0
game_id                   0
game                      0
round                     0
phase                     0
season_code               0
player_id                 0
is_starter                0
is_playing                0
team_id                   0
dorsal                    0
player                    0
minutes                   2
points                    0
two_points_made           0
two_points_attempted      0
three_points_made         0
three_points_attempted    0
free_throws_made          0
free_throws_attempted     0
offensive_rebounds        0
defensive_rebounds        0
total_rebounds            0
assists                   0
steals                    0
turnovers                 0
blocks_favour             0
blocks_against            0
fouls_committed           0
fouls_received            0
valuation                 0
plus_minus                0
dtype: int64


Transform the 'minutes' column (in 'MM:SS' format or 'DNP') into a numerical 'total_seconds' column for quantitative analysis.

In [17]:
# create a new column with total seconds played from 'minutes' column 

# replace empty strings with NaN 
df.loc[:, 'minutes'] = df['minutes'].replace('', pd.NA)
# replace 'DNP' with NaN
df.loc[:, 'minutes'] = df['minutes'].replace('DNP', pd.NA)
# fill missing values with a default value ('00:00')
df.loc[:, 'minutes'] = df['minutes'].fillna('00:00')

# function to convert MM:SS to total seconds (int)
def mm_ss_to_seconds(time_str):
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return (minutes * 60) + seconds  # convert minutes to seconds and add seconds
    except (ValueError, AttributeError):
        return None  # handle invalid or missing values


df.loc[:, 'total_seconds'] = df['minutes'].apply(mm_ss_to_seconds)

Detect duplicate rows based on 'game', 'round', and 'player' to ensure each player’s performance is recorded once per game and round.

In [18]:
# check for duplicate rows
duplicates = df[df.duplicated(subset=['game', 'round', 'player'], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [game_player_id, game_id, game, round, phase, season_code, player_id, is_starter, is_playing, team_id, dorsal, player, minutes, points, two_points_made, two_points_attempted, three_points_made, three_points_attempted, free_throws_made, free_throws_attempted, offensive_rebounds, defensive_rebounds, total_rebounds, assists, steals, turnovers, blocks_favour, blocks_against, fouls_committed, fouls_received, valuation, plus_minus, total_seconds]
Index: []

[0 rows x 33 columns]


Check unique values in 'game', 'phase', and 'player' columns for consistency (e.g., no typos or format variations).

In [26]:
# check unique values for categorical columns to ensure that categorical columns have consistent formats
print(df['game'].unique())
print(df['phase'].unique())
print(df['player'].unique())

['MAD-PAR' 'PAR-MAD' 'ULK-PAR' 'PAR-OLY' 'BAR-PAR' 'ZAL-PAR' 'PAR-BAS'
 'TEL-PAR' 'PAM-PAR' 'MCO-PAR' 'RED-PAR' 'ASV-PAR' 'PAR-TEL' 'PAR-ASV'
 'PAR-PAM' 'PAR-PAN' 'PAR-MUN' 'VIR-PAR' 'PAR-BER' 'PAR-RED' 'BAS-PAR'
 'MUN-PAR' 'OLY-PAR' 'IST-PAR' 'PAN-PAR' 'PAR-ULK' 'PAR-BAR' 'BER-PAR'
 'PAR-MIL' 'PAR-IST' 'PAR-VIR' 'MIL-PAR' 'PAR-ZAL' 'PAR-MCO']
['PLAYOFFS' 'REGULAR SEASON']
['Punter Kevin' 'Vukcevic Tristan' 'Leday Zach' 'Koprivica Balsa'
 'Smailagic Alen' 'Papapetrou Ioannis' 'Lessort Mathias' 'Trifunovic Uros'
 'Madar Yam' 'PARTIZAN MOZZART BET BELGRADE' 'Avramovic Aleksa'
 'Brodziansky Vladimir' 'Glas Gregor' 'Jovanovic Dordje' 'Andjusic Danilo'
 'Nunnally James' 'Exum Dante']


Reformat player names for readability and consistency (e.g., 'LEDAY, ZACH' → 'Leday Zach').

In [20]:
# edit player names for better readability
df['player'] = df['player'].replace({
    'MADAR, YAM' : 'Madar Yam',
    'LEDAY, ZACH' : 'Leday Zach',
    'TRIFUNOVIC, UROS' : 'Trifunovic Uros',
    'NUNNALLY, JAMES WILLIAM' : 'Nunnally James',
    'ANDUSIC, DANILO' : 'Andjusic Danilo',
    'EXUM, DANTE LIMAN' : 'Exum Dante',
    'LESSORT, MATHIAS' : 'Lessort Mathias',
    'PAPAPETROU, IOANNIS' : 'Papapetrou Ioannis',
    'PUNTER, KEVIN  XAVIER' : 'Punter Kevin',
    'VUKCEVIC, TRISTAN' : 'Vukcevic Tristan',
    'SMAILAGIC, ALEN' : 'Smailagic Alen',
    'AVRAMOVIC, ALEKSA' : 'Avramovic Aleksa',
    'GLAS, GREGOR' : 'Glas Gregor',
    'KOPRIVICA, BALSA' : 'Koprivica Balsa',
    'BRODZIANSKY, VLADIMIR' : 'Brodziansky Vladimir',
    'JOVANOVIC, DORDIJE' : 'Jovanovic Dordje',
})

Convert 'is_starter' and 'is_playing' columns from float (1.0, 0.0) to boolean (True, False) for clarity.

In [21]:
# Convert float64 with (1.0 and 0.0 values) to bool (True and False)
df['is_starter'] = df['is_starter'].astype(bool)
df['is_playing'] = df['is_playing'].astype(bool)

Ensure 'total_seconds' and 'is_playing' align logically (e.g., 'total_seconds' == 0 should mean 'is_playing' == False), noting exceptions like technical fouls.

In [22]:
# check for logical inconsistencies 
def check_inconsistencies():
    inconsistencies_is_playing_true = df[(df['total_seconds'] == 0) & (df['is_playing'] == True)]
    inconsistencies_is_playing_false = df[(df['total_seconds'] != 0) & (df['is_playing'] == False)]
    
    if inconsistencies_is_playing_true.empty and inconsistencies_is_playing_false.empty:
        print('No inconsistencies found')
        
    if not inconsistencies_is_playing_true.empty:
        print(inconsistencies_is_playing_true)
        print('Reason: this behavior is consequence of foul committed on the bench or a technical foul') 
    
    if not inconsistencies_is_playing_false.empty:
        print(inconsistencies_is_playing_false)
        print('Reason: this behavior is consequence of foul committed on the bench or a technical foul')
        
check_inconsistencies()

        game_player_id    game_id     game  round     phase season_code  \
189  E2022_313_P009212  E2022_313  MAD-PAR     36  PLAYOFFS       E2022   
406  E2022_313_P000956  E2022_313  MAD-PAR     36  PLAYOFFS       E2022   

    player_id  is_starter  is_playing team_id  ... assists steals turnovers  \
189   P009212       False        True     PAR  ...       0      0         0   
406   P000956       False        True     PAR  ...       0      0         0   

     blocks_favour  blocks_against  fouls_committed  fouls_received  \
189              0               0                1               0   
406              0               0                1               0   

     valuation  plus_minus  total_seconds  
189         -1           0              0  
406         -1           0              0  

[2 rows x 33 columns]
Reason: this behavior is consequence of foul committed on the bench or a technical foul


Confirm all columns have appropriate data types for analysis.

In [23]:
# check data types to ensure that all columns have the correct data type
print(df.dtypes)

game_player_id            object
game_id                   object
game                      object
round                      int64
phase                     object
season_code               object
player_id                 object
is_starter                  bool
is_playing                  bool
team_id                   object
dorsal                    object
player                    object
minutes                   object
points                     int64
two_points_made            int64
two_points_attempted       int64
three_points_made          int64
three_points_attempted     int64
free_throws_made           int64
free_throws_attempted      int64
offensive_rebounds         int64
defensive_rebounds         int64
total_rebounds             int64
assists                    int64
steals                     int64
turnovers                  int64
blocks_favour              int64
blocks_against             int64
fouls_committed            int64
fouls_received             int64
valuation 

Drop irrelevant columns to simplify the dataset for analysis.

In [24]:
# drop unnecessary columns
df = df.drop(columns=['player_id', 'game_player_id', 'game_id', 'season_code', 'team_id', 'dorsal'])

Export the cleaned dataset to CSV and Excel files for future use.

In [25]:
# save the cleaned data to a new csv file
df.to_csv('partizan_2022_cleaned.csv', index=False)
print('Data saved to partizan_2022_cleaned.csv')

# save the cleaned data to a new excel file
df.to_excel('partizan_2022_cleaned.xlsx', index=False)
print('Data saved to partizan_2022_cleaned.xlsx')



Data saved to partizan_2022_cleaned.csv


Data saved to partizan_2022_cleaned.xlsx


## Conclusion
The raw dataset is now cleaned - missing values handled, formats standardized, inconsistencies checked, and unnecessary columns dropped. The cleaned data is saved and ready for validation or analysis.